# Loop 3 Analysis: Fix Submission Overlap Issue

The submission failed with 'Overlapping trees in group 002'. Need to:
1. Validate the correct submission file
2. Understand why the previous submission had overlaps
3. Identify a path forward to beat the target

In [ ]:
import pandas as pd
import numpy as np
from shapely.geometry import Polygon
from shapely import affinity
from shapely.ops import unary_union
import warnings
warnings.filterwarnings('ignore')

# Tree shape definition
TX = [0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125, 0]
TY = [0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5, 0.8]

def get_tree_polygon(x, y, angle_deg):
    coords = list(zip(TX, TY))
    poly = Polygon(coords)
    poly = affinity.rotate(poly, angle_deg, origin=(0, 0))
    poly = affinity.translate(poly, xoff=x, yoff=y)
    return poly

def has_overlap(poly1, poly2, tolerance=1e-9):
    if not poly1.intersects(poly2):
        return False
    intersection = poly1.intersection(poly2)
    return intersection.area > tolerance

def get_side_length(polys):
    union = unary_union(polys)
    bounds = union.bounds
    return max(bounds[2] - bounds[0], bounds[3] - bounds[1])

def check_overlaps(polys):
    for i in range(len(polys)):
        for j in range(i+1, len(polys)):
            if has_overlap(polys[i], polys[j]):
                return True
    return False

print('Functions defined')

In [ ]:
# Load the CORRECT submission file (restored from snapshot)
df = pd.read_csv('/home/submission/submission.csv')
df['x_val'] = df['x'].astype(str).str.replace('s', '').astype(float)
df['y_val'] = df['y'].astype(str).str.replace('s', '').astype(float)
df['deg_val'] = df['deg'].astype(str).str.replace('s', '').astype(float)
df['n'] = df['id'].apply(lambda x: int(str(x).split('_')[0]))

print(f'Loaded {len(df)} rows')
print(f'N values: {df["n"].min()} to {df["n"].max()}')

In [ ]:
# Validate N=2 specifically (the group that had overlaps)
n2_group = df[df['n'] == 2]
print(f'N=2 has {len(n2_group)} trees')
for _, row in n2_group.iterrows():
    print(f"  {row['id']}: x={row['x_val']:.6f}, y={row['y_val']:.6f}, deg={row['deg_val']:.6f}")

# Check for overlaps in N=2
polys = [get_tree_polygon(row['x_val'], row['y_val'], row['deg_val']) for _, row in n2_group.iterrows()]
print(f'\nOverlap check for N=2: {check_overlaps(polys)}')

# Check intersection area
if len(polys) == 2:
    intersection = polys[0].intersection(polys[1])
    print(f'Intersection area: {intersection.area}')

In [ ]:
# Full validation of all N groups
print('Validating all N groups...')
overlap_groups = []
total_score = 0

for n in range(1, 201):
    group = df[df['n'] == n]
    polys = [get_tree_polygon(row['x_val'], row['y_val'], row['deg_val']) for _, row in group.iterrows()]
    
    # Check overlaps
    has_overlap_flag = check_overlaps(polys)
    if has_overlap_flag:
        overlap_groups.append(n)
    
    # Calculate score
    side = get_side_length(polys)
    score = side**2 / n
    total_score += score

print(f'\nTotal groups with overlaps: {len(overlap_groups)}')
if overlap_groups:
    print(f'Groups with overlaps: {overlap_groups[:10]}...')
print(f'\nTotal score: {total_score:.6f}')

In [ ]:
# If the submission is valid, save it properly
if len(overlap_groups) == 0:
    print('Submission is VALID!')
    print(f'Score: {total_score:.6f}')
    print(f'Target: 68.894234')
    print(f'Gap: {total_score - 68.894234:.6f}')
else:
    print(f'INVALID: {len(overlap_groups)} groups have overlaps')